In [1]:
#Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials saved from file
with open('/Users/leratomatlala/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)


# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# Set up the parameters for the API request
Location = 'San Francisco'
Term = 'Pasta'

# Specifying JSON_FILE filename
JSON_FILE = "Data/results_in_progress_SF_Pasta.json"


In [4]:
#Function to add data to json file
def create_json_file(JSON_FILE, delete_if_exists=False):
    file_exists = os.path.isfile(JSON_FILE)

    if file_exists:
        if delete_if_exists:
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            os.remove(JSON_FILE)
            create_json_file(JSON_FILE, delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")
    else:
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
    folder = os.path.dirname(JSON_FILE)
    
    if len(folder) > 0:
        os.makedirs(folder, exist_ok=True)
        
    with open(JSON_FILE, 'w') as f:
        json.dump([], f)

In [5]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=Location,
                                term=Term,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



[!] Data/results_in_progress_SF_Pasta.json already exists. Deleting previous file...
[i] Data/results_in_progress_SF_Pasta.json not found. Saving empty list to new file.
- 0 previous results found.


95

In [7]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=Location,
                                    term=Term, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)



  0%|          | 0/95 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [8]:
# load the results
Pasta_Locations_df = pd.read_json(JSON_FILE)
display(Pasta_Locations_df.head(), Pasta_Locations_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,oz2PZG07x0JBxBFZ07zyzA,pasta-supply-co-san-francisco,Pasta Supply Co,https://s3-media2.fl.yelpcdn.com/bphoto/-btmf0...,False,https://www.yelp.com/biz/pasta-supply-co-san-f...,110,"[{'alias': 'pastashops', 'title': 'Pasta Shops'}]",4.5,"{'latitude': 37.78331, 'longitude': -122.4618}",[],$$,"{'address1': '236 Clement St', 'address2': Non...",+14152362668,(415) 236-2668,3345.094765
1,msT3LrLB4fhN04HYHuFsew,bella-trattoria-san-francisco,Bella Trattoria,https://s3-media2.fl.yelpcdn.com/bphoto/lDIjLb...,False,https://www.yelp.com/biz/bella-trattoria-san-f...,1882,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 37.78136, 'longitude': -122.46092...","[delivery, pickup]",$$,"{'address1': '3854 Geary Blvd', 'address2': ''...",+14152210305,(415) 221-0305,3131.066552
2,GUMD4UYvQ01RLcvQP74A-g,pasta-panino-san-francisco,Pasta Panino,https://s3-media1.fl.yelpcdn.com/bphoto/-FKpb9...,False,https://www.yelp.com/biz/pasta-panino-san-fran...,51,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",5.0,"{'latitude': 37.761014, 'longitude': -122.435911}",[],$$,"{'address1': '4150 18th St', 'address2': '', '...",+14158293829,(415) 829-3829,48.215609
3,QueFVMcMlT-6aZFv2M47mg,bottega-san-francisco-2,Bottega,https://s3-media2.fl.yelpcdn.com/bphoto/IawDcF...,False,https://www.yelp.com/biz/bottega-san-francisco...,920,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 37.75472, 'longitude': -122.4212}","[delivery, pickup]",$$,"{'address1': '1132 Valencia St', 'address2': N...",+14156559048,(415) 655-9048,1506.250343
4,FRpULkKmvD9caSKabQzq5w,the-italian-homemade-company-san-francisco-6,The Italian Homemade Company,https://s3-media4.fl.yelpcdn.com/bphoto/gU5tMP...,False,https://www.yelp.com/biz/the-italian-homemade-...,1227,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 37.79742, 'longitude': -122.43097}","[delivery, pickup]",$$,"{'address1': '1919 Union St', 'address2': '', ...",+14156255965,(415) 625-5965,4105.765157


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,SEuHSFDOoux4MqusTQPNMw,johns-grill-san-francisco,John's Grill,https://s3-media4.fl.yelpcdn.com/bphoto/G1_mMR...,False,https://www.yelp.com/biz/johns-grill-san-franc...,1624,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",3.5,"{'latitude': 37.78545339842801, 'longitude': -...",[delivery],$$$,"{'address1': '63 Ellis St', 'address2': '', 'a...",+14159860069,(415) 986-0069,3749.995677
996,P1P-ByAIL4OlqlBTzrVlhg,murphys-pub-san-francisco,Murphy's Pub,https://s3-media1.fl.yelpcdn.com/bphoto/PnytFy...,False,https://www.yelp.com/biz/murphys-pub-san-franc...,430,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",3.0,"{'latitude': 37.79019126223643, 'longitude': -...",[delivery],$$,"{'address1': '217 Kearny St', 'address2': '', ...",+14156939588,(415) 693-9588,4323.902335
997,AA1wdUC-d3XyHp_6G2dD_w,morning-due-cafe-san-francisco,Morning Due Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/ghTfjy...,False,https://www.yelp.com/biz/morning-due-cafe-san-...,593,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.0,"{'latitude': 37.7630481, 'longitude': -122.428...","[delivery, pickup]",$$,"{'address1': '3698 17th St', 'address2': '', '...",+14156217156,(415) 621-7156,744.439547
998,qnU16TsePK94mLsRe6t3Yw,mariposa-baking-san-francisco,Mariposa Baking,https://s3-media3.fl.yelpcdn.com/bphoto/2dLiZe...,False,https://www.yelp.com/biz/mariposa-baking-san-f...,475,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.0,"{'latitude': 37.7954363868498, 'longitude': -1...",[],$$,"{'address1': '1 Ferry Bldg', 'address2': 'Ste ...",+15105950955,(510) 595-0955,5375.480627
999,sOQRZ2riez4mMdzA_gGUGQ,crepevine-restaurants-berkeley-3,Crepevine Restaurants,https://s3-media2.fl.yelpcdn.com/bphoto/nTU7vh...,False,https://www.yelp.com/biz/crepevine-restaurants...,917,"[{'alias': 'newamerican', 'title': 'New Americ...",3.5,"{'latitude': 37.87811, 'longitude': -122.269401}","[delivery, pickup]",$$,"{'address1': '1600 Shattuck Ave', 'address2': ...",+15107051836,(510) 705-1836,19622.756178


In [9]:
# check for duplicate ID's 
Pasta_Locations_df.duplicated(subset='id').sum()

0